# Script ready for final acceptance testing

# Restart the cache

In [ ]:
from pynq import PL
PL.reset()

In [ ]:
from pynq.overlays.AXIS_Adder import AXIS_Adder

beamforming = AXIS_Adder('AXIS_Adder.bit')
beamforming.init_rf_clks()

# Inspect beamforming overlay hierarchies and IP

# Initialize ADCs and number of samples

In [ ]:
import xrfdc
# ADC initialization
# Channels
DAC_CHANNEL_B = 0 # 'Channel 0': {'Tile': 224, 'Block': 0}
DAC_CHANNEL_A = 1 # 'Channel 1': {'Tile': 230, 'Block': 0}

ADC_CHANNEL_D = 0 # 'Channel 0': {'Tile': 224, 'Block': 0} ch00
ADC_CHANNEL_C = 1 # 'Channel 1': {'Tile': 224, 'Block': 1} ch01
ADC_CHANNEL_B = 2 # 'Channel 2': {'Tile': 226, 'Block': 0} ch20
ADC_CHANNEL_A = 3 # 'Channel 3': {'Tile': 226, 'Block': 1} ch21

# Must be in reverse order (i.e. C must come before B which must come before A)
adc_array = [ADC_CHANNEL_D, ADC_CHANNEL_C, ADC_CHANNEL_B, ADC_CHANNEL_A]

adc_char_array = ['D', 'C', 'B', 'A']

sample_scaler = 2048
number_samples = int(32768/sample_scaler)  # Between 16 and 32768
decimation_factor = 1 # 2 is default
sample_frequency = 4915.2e6/decimation_factor  # Hz The default sample frequency is 4915.2e6 Hz which is sufficient for our signal

original_adc_settings = beamforming.radio.receiver.channel[ADC_CHANNEL_D].adc_block.MixerSettings

# for ADC in adc_array:
#     beamforming.radio.receiver.channel[ADC].adc_block.DecimationFactor = decimation_factor
#     beamforming.radio.receiver.channel[ADC].adc_block.MixerSettings = {
#         'CoarseMixFreq':  xrfdc.COARSE_MIX_BYPASS,
#         'EventSource':    xrfdc.EVNT_SRC_TILE, 
#         'FineMixerScale': xrfdc.MIXER_SCALE_1P0,
#         'Freq':           1, #fs/4
#         'MixerMode':      xrfdc.MIXER_MODE_R2C,
#         'MixerType':      xrfdc.MIXER_TYPE_FINE,
#         'PhaseOffset':    0.0
#     }
#     beamforming.radio.receiver.channel[ADC].adc_block.UpdateEvent(xrfdc.EVENT_MIXER)
    
print("Original ADC settings:", original_adc_settings)
#print("New ADC settings:", beamforming.radio.receiver.channel[ADC].adc_block.MixerSettings)

# Calculate beamforming weights

In [ ]:
import numpy as np
d = 0.5 # half wavelength spacing
Nr = len(adc_array)
steering_angle_degrees = int(input("Input the steering angle in degrees:"))
steering_angle = steering_angle_degrees / 180 * np.pi # convert to radians
if steering_angle_degrees > 0:
    beamforming_weights = np.exp(-2j * np.pi * d * np.arange(Nr) * np.sin(steering_angle)) # array factor
else:
    beamforming_weights = np.exp(-2j * np.pi * d * np.arange(Nr)[ : :-1] * np.sin(steering_angle)) # array factor
print("beamforming_weights:", beamforming_weights)

# Pass them into GPIO

In [ ]:
from pynq.lib import AxiGPIO

def float_to_8bit(weight):
    # Extract and shift from the range of -1 to 1 to 0 to 2
    #real_shifted = weight + 1
    
    # Integer scale to the range of -127 to 127
    real_scaled = weight * 127/2
    
    # Round the scaled real part to the nearest whole number
    real_rounded = np.round(real_scaled)
    
    # Clip the rounded value to ensure it is within the -127 to 127 range (Just a safegaurd)
    real_clipped = np.clip(real_rounded, -127, 127)
    
    # Convert the clipped value to a signed 8-bit integer
    real_8bit = real_clipped.astype(np.int8)
    
    return real_8bit

def from_8bit_to_complex(real_8bit, imag_8bit):
    # This is what will be done after the 8 bit ints are received through the gpios
    real = real_8bit / (127/2)
    imag = imag_8bit / (127/2)
    return complex(real, imag)
    

real_ip = beamforming.ip_dict['bWeights_real'] # real_bWeights
imag_ip = beamforming.ip_dict['bWeights_imag'] # imag_bWeights

real = AxiGPIO(real_ip).channel1
imag = AxiGPIO(imag_ip).channel1

zeroed_weights = []
for ADC in adc_array:
    zeroed_weights.append(0+0j)
    
#beamforming_weights = zeroed_weights

b_real = 0
b_imag = 0
for ADC in adc_array:
    if ADC == 0:
        shift = 0
    elif ADC == 1:
        shift = 8
    elif ADC == 2:
        shift = 16
    elif ADC == 3:
        shift = 24

    real_part = float_to_8bit(np.real(beamforming_weights[ADC]))
    imag_part = float_to_8bit(np.imag(beamforming_weights[ADC]))

    b_real = b_real | real_part << shift
    b_imag = b_imag | imag_part << shift

    test_ = from_8bit_to_complex(real_part, imag_part)
    print("Conversion errors for beamforming weight for ADC", adc_char_array[ADC])
    print("Real part:", 100*abs(1-(np.real(test_)/np.real(beamforming_weights[ADC]))), "%")
    if np.imag(beamforming_weights[ADC]) == 0:
        print("Imag part is", np.imag(test_), "off of anticipated value of 0.")
    else:
        print("Imag part:", 100*abs(1-(np.imag(test_)/np.imag(beamforming_weights[ADC]))), "%")

print("b_real:", int(b_real))
print("b_imag:", int(b_imag))
real.write(0xffffffff, 0x00000000)
imag.write(0xffffffff, 0x00000000)
real.write(0xffffffff, int(b_real))
imag.write(0xffffffff, int(b_imag))

print(beamforming.bWeights_real.register_map)
print(beamforming.bWeights_imag.register_map)

# Transfer beamformed data

In [ ]:
#beamforming.radio.receiver.channel_00.axi_dma_real.recvchannel.start()
#beamforming.radio.receiver.channel_00.axi_dma_imag.recvchannel.start()

In [ ]:
import pandas as pd
# writer = pd.ExcelWriter('ADC_Data.xlsx', engine='xlsxwriter')
beamformed_data = []
# Something like this:
for ADC in adc_array: 
    beamformed_data.append(beamforming.radio.receiver.channel[ADC].transfer(number_samples))
data = pd.DataFrame(beamformed_data)
data.to_excel("ADC_Data.xlsx")

print(beamforming.radio.receiver.channel_00.axi_dma_real.register_map)

# Plot beamformed data
Also add code to calculate max power

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.interpolate import CubicSpline
import numpy as np

beamformed_interpolated_data = []

# Create Plotly figure for interpolated ADC data
beamformed_fig = make_subplots(specs=[[{"secondary_y": False}]])  # Adjust as necessary

# Needed for interpolation

time_data = np.arange(0, number_samples/sample_frequency, 1/sample_frequency)
dense_t = np.linspace(time_data.min(), time_data.max(), len(time_data) * 10)  # Increase density

for ADC in adc_array:
    sampled_signal = np.real(beamformed_data[ADC])
    cs_real = CubicSpline(time_data, sampled_signal)

    # Beamformed interpolated data
    beamformed_interpolated_data.append(cs_real(dense_t))

    # Add beamformed interpolated data trace
    beamformed_fig.add_trace(
    go.Scatter(x=dense_t, y=beamformed_interpolated_data[ADC], name=f"ADC " + adc_char_array[ADC] + " Data"),
    secondary_y=False,
    )

# Update layout of raw ADC data
#beamformed_fig.update_yaxes(title_text="yaxis title", range=[-1, 1], row=1, col=1)
beamformed_fig.update_layout(
title=f"Time Domain Plot of ADC Channels",
xaxis_title="Time(s)",
yaxis_title="Amplitude(V)",
)

# Show beamformed ADC data
beamformed_fig.show()  
print(beamforming.radio.receiver.channel_00.axi_dma_imag.register_map)